Reference: https://towardsdatascience.com/web-scraping-indeed-com-e5591790736d

Reference:
https://github.com/rowandl/portfolio/blob/master/Webscraping%20Indeed.com/Webscraping%20Indeed.ipynb

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
#!pip install pandas
#!pip install bs4
#!pip install matplotlib
#!pip install lxml

In [24]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup
import csv
import time

from IPython.display import HTML

In [25]:
# top 10 canada cities
#cities = ['Toronto', 'Montreal', 'Vancouver', 'Calgary', 'Edmonton', 'Ottawa', 'Winnipeg', 'Quebec+City', 'Hamilton', 'Kitchener']
#cities = ['Toronto']

# Set this to a high-value to generate more results. High value might get block by website
#max_results_per_city = 100
#max_results_per_city = 10
max_start = 100

#total_count = len(cities) * max_results_per_city
#city_count = 0
job_count = 1

#making a dataframe which collects all the information from my webscraping
#https://www.indeed.ca/jobs?q=data+scientist+%2420%2C000&l=Toronto&start=0
url_start = "https://ca.indeed.com/jobs?q=data+scientist"
url_template = "https://www.indeed.ca/jobs?q=data+scientist&start={}"
#url_template = "https://www.indeed.ca/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
base_url = "https://ca.indeed.com"

df = pd.DataFrame(columns=["location", 'company', 'job_title', 'salary', 'job_description'])

start = 0

while start < max_start:
    url = url_template.format(start)
    url = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, "html.parser")

    try:
        max_search = [int(s) for s in soup.find('div', attrs = {'id': 'searchCountPages'}).text.split() if s.isdigit()][1]
    except:
        pass

    if (max_start < max_search):
        max_start = max_search

    for b in soup.find_all('div', attrs = {'class': 'result'}):
        try:
            location = b.find('div', attrs = {'class': 'location'}).text
        except:
            try:
                location = b.find('span', attrs = {'class': 'location'}).text
            except:
                location = 'NA'

        try:
            job_title = b.find('a', attrs = {'data-tn-element':'jobTitle'}).text
            job_title = job_title.replace("\n","")
        except:
            job_title = 'NA'
        
        try:
            job_url = b.find('a', attrs = {'data-tn-element':'jobTitle'}).get('href')
        except:
            job_url = 'NA'

        try:
            company = b.find('span', attrs = {'class':'company'}).text
            company = company.replace("\n","")
        except:
            company = 'NA'

        try:
            salary = b.find('span', attrs = {'class' : 'salaryText'}).text  
        except:
            salary = 'NA'
            
        #time.sleep(1) # to prevent web blocking

        if (job_url != 'NA'):
            url_job = 'https://ca.indeed.com' + job_url
            url_job = urllib.request.Request(url_job, headers={'User-Agent': 'Mozilla/5.0'})
            html_job = urllib.request.urlopen(url_job).read()
            soup_job = BeautifulSoup(html_job, "html.parser")

            try:
                description = soup_job.find('div', attrs = {'id': 'jobDescriptionText'}).text
            except:
                description = 'NA'
        else:
            description = 'NA'

        done = int((job_count / max_start) * 100)

        print("%d%% - %d / %d: %s > %s > %s > %s" % (done, job_count, max_start, location, company, salary, job_title))

        if (description != 'NA'):
            df = df.append({"location":location,"company":company, "job_title": job_title, "salary": salary, "job_description": description}, ignore_index=True)
            
        #time.sleep(1) # to prevent web blocking
        job_count += 1

    start += 10
   
df.drop_duplicates(inplace=True)
df.company.replace(inplace=True,regex=True,to_replace="\n",value="")
df.job_title.replace(inplace=True,regex=True,to_replace="\n",value="")
df.head()

0% - 1 / 679: Mississauga, ON > Hatch > NA > Data Scientist
0% - 2 / 679: Toronto, ON > Loopio > NA > Data Scientist
0% - 3 / 679: Waterloo, ON > McAfee > NA > Data Scientist
0% - 4 / 679: Montréal, QC > EXFO > NA > Data Scientist
0% - 5 / 679: Montréal, QC > Canadian National Railway > NA > Data Scientist
0% - 6 / 679: Oakville, ON > Geotab > NA > Data Scientist
1% - 7 / 679: Kitchener, ON > Oracle > NA > Data Scientist
1% - 8 / 679: Vancouver, BC > Softvision - North America > NA > Data Scientist
1% - 9 / 679: Montréal, QC > Canadian National Railway > NA > Specialist System Design, Data Scientist Machine Learning an...
1% - 10 / 679: Quebec City, QC > Gartner > NA > Specialist, Data Science and Machine Learning Development
1% - 11 / 679: Mississauga, ON > Compass Digital Labs > NA > Data Scientist (Decision Science)
1% - 12 / 679: Moncton, NB > Fiddlehead > NA > Data Scientist
1% - 13 / 679: Winnipeg, MB > Manitoba Government > 
$67,671 - $82,064 a year > Senior Data Scientist
2% - 

,location,company,job_title,salary,job_description
0,"Mississauga, ON",Hatch,Data Scientist,NA,Requisition ID: 33710\nJob Category: Informati...
1,"Toronto, ON",Loopio,Data Scientist,NA,Loopio is a workplace that unleashes learning ...
2,"Waterloo, ON",McAfee,Data Scientist,NA,"Job Title:\nData Scientist\n\nLocation:\nUS, T..."
3,"Montréal, QC",EXFO,Data Scientist,NA,Job title: Data Scientist\nDivision/function: ...
4,"Montréal, QC",Canadian National Railway,Data Scientist,NA,Job Summary\nThe Data Scientist is responsible...


In [26]:
df

,location,company,job_title,salary,job_description
0,"Mississauga, ON",Hatch,Data Scientist,NA,Requisition ID: 33710\nJob Category: Informati...
1,"Toronto, ON",Loopio,Data Scientist,NA,Loopio is a workplace that unleashes learning ...
2,"Waterloo, ON",McAfee,Data Scientist,NA,"Job Title:\nData Scientist\n\nLocation:\nUS, T..."
3,"Montréal, QC",EXFO,Data Scientist,NA,Job title: Data Scientist\nDivision/function: ...
4,"Montréal, QC",Canadian National Railway,Data Scientist,NA,Job Summary\nThe Data Scientist is responsible...
...,...,...,...,...,...
549,"Toronto, ON",nugget.ai,Machine Learning Engineer,NA,nugget.ai is recruiting and screening candidat...
580,"Montréal, QC",Unity Technologies,Senior Data Scientist/Scientifique de données ...,NA,The Technology organization is seeking a Senio...
582,"Toronto, ON",Kinaxis,Business Data Scientist (Professional Services),NA,"2\n\nAt Kinaxis, who we are is grounded in our..."
583,"Montréal, QC",Lightspeed POS,Senior Data Analyst,NA,Lightspeed is seeking a full-time Senior Data ...


In [27]:
from datetime import datetime

# sending it to csvs to save the data
now = datetime.now()
dt_string = now.strftime("%y%m%d%H%M")
filename = "Indeed_" + dt_string + ".csv"
df.to_csv(filename , sep=',', encoding='utf-8')

In [28]:
%cp *.csv /content/gdrive/MyDrive

In [29]:
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>